<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


- Install the additional package requirements for this bonus notebook by uncommenting and running the following cell:

In [31]:
# pip install -r requirements-extra.txt

# Comparing Various Byte Pair Encoding (BPE) Implementations

<br>
&nbsp;

## Using BPE from `tiktoken`

In [28]:
from importlib.metadata import version

print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.9.0


In [27]:
import tiktoken

tik_tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, world. Is this-- a test?"

In [26]:
integers = tik_tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [25]:
strings = tik_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


In [24]:
print(tik_tokenizer.n_vocab)

50257


<br>
&nbsp;

## Using the original BPE implementation used in GPT-2

In [23]:
from bpe_openai_gpt2 import get_encoder, download_vocab

In [22]:
download_vocab()

Fetching encoder.json: 1.04Mit [00:00, 3.80Mit/s]                                                   
Fetching vocab.bpe: 457kit [00:00, 2.07Mit/s]                                                       


In [21]:
orig_tokenizer = get_encoder(model_name="gpt2_model", models_dir=".")

In [20]:
integers = orig_tokenizer.encode(text)

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [19]:
strings = orig_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


<br>
&nbsp;

## Using the BPE via Hugging Face transformers

In [54]:
import transformers

transformers.__version__

ModuleNotFoundError: No module named 'transformers'

In [34]:
from transformers import GPT2Tokenizer

hf_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

ModuleNotFoundError: No module named 'transformers'

In [35]:
hf_tokenizer(strings)["input_ids"]

NameError: name 'hf_tokenizer' is not defined

In [36]:
from transformers import GPT2TokenizerFast

hf_tokenizer_fast = GPT2TokenizerFast.from_pretrained("gpt2")

ModuleNotFoundError: No module named 'transformers'

In [37]:
hf_tokenizer_fast(strings)["input_ids"]

NameError: name 'hf_tokenizer_fast' is not defined

<br>
&nbsp;

## Using my own from-scratch BPE tokenizer

In [38]:
import os
import sys
import io
import nbformat
import types

def import_from_notebook():
    def import_definitions_from_notebook(fullname, names):
        current_dir = os.getcwd()
        path = os.path.join(current_dir, "..", "05_bpe-from-scratch", fullname + ".ipynb")
        path = os.path.normpath(path)

        # Load the notebook
        if not os.path.exists(path):
            raise FileNotFoundError(f"Notebook file not found at: {path}")

        with io.open(path, "r", encoding="utf-8") as f:
            nb = nbformat.read(f, as_version=4)

        # Create a module to store the imported functions and classes
        mod = types.ModuleType(fullname)
        sys.modules[fullname] = mod

        # Go through the notebook cells and only execute function or class definitions
        for cell in nb.cells:
            if cell.cell_type == "code":
                cell_code = cell.source
                for name in names:
                    # Check for function or class definitions
                    if f"def {name}" in cell_code or f"class {name}" in cell_code:
                        exec(cell_code, mod.__dict__)
        return mod

    fullname = "bpe-from-scratch"
    names = ["BPETokenizerSimple"]

    return import_definitions_from_notebook(fullname, names)

In [39]:
imported_module = import_from_notebook()
BPETokenizerSimple = getattr(imported_module, "BPETokenizerSimple", None)

tokenizer_gpt2 = BPETokenizerSimple()
tokenizer_gpt2.load_vocab_and_merges_from_openai(
    vocab_path=os.path.join("gpt2_model", "encoder.json"),
    bpe_merges_path=os.path.join("gpt2_model", "vocab.bpe")
)

In [40]:
integers = tokenizer_gpt2.encode(text)

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


<br>
&nbsp;

## A quick performance benchmark

In [41]:
with open("../01_main-chapter-code/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

### Original OpenAI GPT-2 tokenizer

In [53]:
%timeit orig_tokenizer.encode(raw_text)

7.97 ms ± 318 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Tiktoken OpenAI GPT-2 tokenizer

In [52]:
%timeit tik_tokenizer.encode(raw_text)

2.35 ms ± 999 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Hugging Face OpenAI GPT-2 tokenizer

In [51]:
%timeit hf_tokenizer(raw_text)["input_ids"]

NameError: name 'hf_tokenizer' is not defined

In [49]:
%timeit hf_tokenizer(raw_text, max_length=5145, truncation=True)["input_ids"]

NameError: name 'hf_tokenizer' is not defined

In [50]:
%timeit hf_tokenizer_fast(raw_text)["input_ids"]

NameError: name 'hf_tokenizer_fast' is not defined

In [16]:
%timeit hf_tokenizer_fast(raw_text, max_length=5145, truncation=True)["input_ids"]

NameError: name 'hf_tokenizer_fast' is not defined

### My own GPT-2 tokenizer (for educational purposes)

In [15]:
%timeit tokenizer_gpt2.encode(raw_text)

NameError: name 'tokenizer_gpt2' is not defined